In [1]:
# Import main librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# For images
from IPython.display import Image

# Ignore warning parts
import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv('data_clustering.csv', sep=';')

In [15]:
orders = pd.read_csv('olist_orders_dataset.csv', sep=',')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv', sep=',')
order_items = pd.read_csv('olist_order_items_dataset.csv', sep=',')
customers = pd.read_csv('olist_customers_dataset.csv', sep=',')

In [20]:
# Create dataframe 'orders_order_reviews'
orders_order_reviews = pd.merge(orders[['order_id',
                                        'customer_id',
                                        'order_purchase_timestamp']],
                               order_reviews[['review_id',
                                              'order_id',
                                              'review_score',
                                             'review_creation_date']])

In [21]:
# We keep only the most recent review for each order_id ('order_id')
orders_order_reviews = orders_order_reviews.sort_values(by = 'review_creation_date', 
                                                        ascending = False)
orders_order_reviews = orders_order_reviews.drop_duplicates(subset = ['order_id'],
                                                            keep ='first')

In [23]:
# Merge datasets
orders_customers = pd.merge(orders_order_reviews[['order_id',
                                                  'customer_id',
                                                  'order_purchase_timestamp',
                                                 'review_id', 
                                                  'review_score']],
                           customers[['customer_id',
                                      'customer_unique_id']])

orders_customers = pd.merge(orders_customers,
                            order_items[['order_id',
                                         'order_item_id',
                                         'product_id',
                                         'price']])

In [25]:
from datetime import datetime, date, time

# Change dtypes 'object' to 'datetime'
orders_customers['order_purchase_timestamp'] = pd.to_datetime(orders_customers['order_purchase_timestamp'])

# Calculate duration between today and order_purchase_timestamp
orders_customers['last_order'] = (datetime.now() - orders_customers['order_purchase_timestamp'])

# Keep number of days only
orders_customers['last_order_days'] = orders_customers['last_order'].astype(str).str[:4].astype(int)

In [31]:
# Creating dataframe with no duplicates (1 product/1 purchage for each customer): 
# Dropping all 'customer_unique_id" duplicates
df_no_duplicates = orders_customers.drop_duplicates(subset = ['customer_unique_id'],
                                                    keep = False)
df_no_duplicates.shape

(81157, 11)

In [32]:
# Creating dataframe only with duplicates (several products and several purchases)
df_duplicates = orders_customers.loc[orders_customers['customer_unique_id'].duplicated(keep = False) == True]
df_duplicates.shape

(28161, 11)

In [34]:
# Change names of features
df_no_duplicates = df_no_duplicates.rename(columns={'customer_unique_id' : 'Customer',
                                                    'price' : 'payments_total',
                                                    'review_score' : 'review_score_mean'})
# Create new feature : number of purchases
df_no_duplicates['number_of_purchases'] = 1

In [59]:
# Select the date of the most recent purchase

# Sort by date
df_last_order = df_duplicates.sort_values('order_purchase_timestamp',
                                          ascending = False)

# Keep the line of the customer with the most recent date
df_last_order = df_last_order.drop_duplicates(subset = ['customer_unique_id'],
                                   keep = 'first')

df_last_order = df_last_order.rename(columns={'customer_unique_id' : 'Customer'})

df = df_last_order[['Customer',
                    'order_purchase_timestamp',
                    'last_order_days']]

In [62]:
# Calculate payments_total for each customer

# Save index for the column "price" in dataframe df_duplicates
index_price = df_duplicates.columns.get_loc('price')
# Create list of customer_unique_id
list_customer_unique_id = df_duplicates['customer_unique_id'].unique().tolist()

# Create new column "payments_total" in df
df["payments_total"] = 0
# Save index for the column "payments_total" in dataframe df
index_payments_total = df.columns.get_loc('payments_total')

# Reset index of dataset df_duplicates
df_duplicates = df_duplicates.reset_index(drop = True)
# Reset index of dataset df
df = df.reset_index(drop = True)

for customer in list_customer_unique_id:
    # Save list of indexes corresponding to the same customer in df_duplicates
    index_list = df_duplicates.loc[df_duplicates['customer_unique_id'] == customer].index
    # Save index of customer in df
    index_customer = df.loc[df['Customer'] == customer].index
    # Add prices for each customers to obtain 'payment_total'
    df.iloc[index_customer, index_payments_total] = df_duplicates.iloc[index_list, index_price].sum()

In [65]:
# Calculate number of purchases for each customer

# Create list of customer_unique_id
list_customer_unique_id = df_duplicates['customer_unique_id'].unique().tolist()

# Create new column "number_of_purchases" in df
df["number_of_purchases"] = 0
# Save index for the column "number_of_purchases" in dataframe df
index_number_of_purchases = df.columns.get_loc('number_of_purchases')

for customer in list_customer_unique_id:
    # Save index of customer in df
    index_customer = df.loc[df['Customer'] == customer].index
    # Create dataframe for each customer
    df_customer = df_duplicates.loc[df_duplicates['customer_unique_id'] == customer]
    # Add number_of_purchases for each customers
    df.iloc[index_customer, index_number_of_purchases] = len(df_customer['order_id'].unique().tolist())

In [66]:
# Calculate review_score_mean for each customer

# Create list of customer_unique_id
list_customer_unique_id = df_duplicates['customer_unique_id'].unique().tolist()

# Create new column "review_score_mean" in df
df["review_score_mean"] = 0
# Save index for the column "review_score_mean" in dataframe df
index_review_score_mean = df.columns.get_loc('review_score_mean')

for customer in list_customer_unique_id:
    # Save index of customer in df
    index_customer = df.loc[df['Customer'] == customer].index
    # Create dataframe for each customer
    df_customer = df_duplicates.loc[df_duplicates['customer_unique_id'] == customer]        
    # Calculate the mean of review_scores for each customers to obtain 'review_score_mean'
    df.iloc[index_customer, index_review_score_mean] = df_customer['review_score'].mean()

In [73]:
# Concatenate dataframes keeping only common features
data = pd.concat([df_no_duplicates, df], join = "inner", ignore_index=True)

In [74]:
data

,order_purchase_timestamp,review_score_mean,Customer,payments_total,last_order_days,number_of_purchases
0,2018-08-14 12:28:49,4.0,0dab841685c250499b36d81dbb570ab8,70.00,1287,1
1,2018-08-12 20:52:56,5.0,27bbdb1c8c1324bfb03ad56475bbfb83,100.00,1288,1
2,2018-08-08 21:30:18,5.0,b65db3f838c1bdb9bbab3fd56b9d6977,32.00,1292,1
3,2018-08-15 14:29:08,1.0,ec979208947bbba310f2ad8e50963725,39.90,1286,1
4,2018-08-10 18:50:46,3.0,9e7f77462df10081f5a74cf47f88208d,199.99,1290,1
...,...,...,...,...,...,...
92650,2016-10-04 16:41:59,1.0,2e15add79c76241beffe812a24ad3a31,129.80,1965,1
92651,2016-10-04 15:02:53,1.0,a9530b74fe56ca4be9e5f2c7e7c4175a,119.80,1966,1
92652,2016-10-04 13:11:29,5.0,f176923a0a4ab546c7287791ccb82193,47.80,1966,1
92653,2016-09-15 12:16:38,1.0,830d5b7aaa3b6f1e9ad63703bec97d23,134.97,1985,1


In [75]:
data0 = data[['Customer',
             'number_of_purchases',
             'payments_total',
             'last_order_days',
             'review_score_mean']]

data0 = data0.set_index('Customer')

In [76]:
X0 = data0.values
X0.shape

(92655, 4)

In [ ]:
data = data.loc[data['order_purchase_timestamp'] < '2018-08-16']

In [ ]:
data = data[['Customer',
             'number_of_purchases',
             'payments_total',
             'last_order_days',
             'review_score_mean']]

data = data.set_index('Customer')

In [ ]:
X = data.values
X.shape

In [ ]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X0)
X_scaled = std_scale.transform(X)

In [ ]:
def filter_date(date, std_scale):
    